# Goal with this notebook is to figure out how to make the dictionaries for novel

In [1]:
import pandas as pd
import numpy as np
import ast
from pathlib import Path
import multirecording_spikeanalysis as spike

In [2]:
cols = ['condition ', 'session_dir', 'all_subjects', 'tone_start_timestamp', 'tone_stop_timestamp']

# Load the data
df = pd.read_excel('rce_pilot_2_per_video_trial_labels.xlsx', usecols=cols, engine='openpyxl')

df2 = df.dropna() # Drop the rows missing data
df3 = df2.copy()
df3['all_subjects'] = df3['all_subjects'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x) # Make the 'all_subjects' column readable as a list
df4 = df3[df3['all_subjects'].apply(lambda x: len(x) < 3)] # Ignore novel sessions for now

# Initialize an empty list to collect data for the new DataFrame
new_df_data = []

for _, row in df4.iterrows():
    session_dir = row['session_dir']
    subjects = row['all_subjects']
    condition = row['condition ']

    # Split session_dir on '_subj_' and take the first part only
    # This ensures everything after '_subj_' is ignored
    base_session_dir = session_dir.split('_subj_')[0]

    for subject in subjects:
        subject_formatted = subject.replace('.', '-')
        # Append formatted subject to the base session_dir correctly
        subj_recording = f"{base_session_dir}_subj_{subject_formatted}"
        new_df_data.append({
            'session_dir': session_dir,
            'subject': subject,
            'subj_recording': subj_recording,
            'condition': condition if condition in ['rewarded', 'omission', 'both_rewarded', 'tie'] else ('win' if str(condition) == str(subject) else 'lose'),
            'tone_start_timestamp': row['tone_start_timestamp'],
            'tone_stop_timestamp': row['tone_stop_timestamp']
        })

# Convert list to DataFrame
new_df = pd.DataFrame(new_df_data)
new_df = new_df.drop_duplicates()

In [3]:
# Prepare timestamp_dicts from new_df
timestamp_dicts = {}
for _, row in new_df.iterrows():
    key = row['subj_recording']
    condition = row['condition']
    timestamp_start = int(row['tone_start_timestamp']) // 20
    timestamp_end = int(row['tone_stop_timestamp']) // 20
    tuple_val = (timestamp_start, timestamp_end)

    if key not in timestamp_dicts:
        timestamp_dicts[key] = {cond: [] for cond in ['rewarded', 'win', 'lose', 'omission', 'both_rewarded', 'tie']}
    timestamp_dicts[key][condition].append(tuple_val)

# Convert lists in timestamp_dicts to numpy arrays
for subj_recording in timestamp_dicts:
    for condition in timestamp_dicts[subj_recording]:
        timestamp_dicts[subj_recording][condition] = np.array(timestamp_dicts[subj_recording][condition], dtype=np.int64)

In [8]:
# Construct the path in a platform-independent way (HiPerGator or Windows)
ephys_path = Path('.') / 'recordings' / 'updated_phys' / 'novel'

ephys_data = spike.EphysRecordingCollection(str(ephys_path))

<class 'numpy.ndarray'>
20230628_111202_standard_comp_to_novel_agent_D1_subj_1-1vs1-2and2-2_merged.rec
<class 'numpy.ndarray'>
20230628_111202_standard_comp_to_novel_agent_D1_subj_1-2vs1-1and2-1_merged.rec
<class 'numpy.ndarray'>
20230629_111937_standard_comp_to_novel_agent_D2_subj_1-1v1-4and2-1_merged.rec
<class 'numpy.ndarray'>
20230629_111937_standard_comp_to_novel_agent_D2_subj_1-4vs1-1and2-2_merged.rec
<class 'numpy.ndarray'>
20230630_115506_standard_comp_to_novel_agent_D3_subj_1-4vs1-2and2-1_merged_merged.rec
Please assign event dictionaries to each recording
as recording.event_dict
event_dict = {event name(str): np.array[[start(ms), stop(ms)]...]
Please assign subjects to each recording as recording.subject


In [9]:
for recording in ephys_data.collection.keys():
    # Check if the recording key (without everything after subject #) is in timestamp_dicts
    start_pos = recording.find('subj_')
    # Add the length of 'subj_' and 3 additional characters to include after 'subj_'
    end_pos = start_pos + len('subj_') + 3
    # Slice the recording key to get everything up to and including the subject identifier plus three characters
    recording_key_without_suffix = recording[:end_pos]
    if recording_key_without_suffix in timestamp_dicts:
        # Assign the corresponding timestamp_dicts dictionary to event_dict
        ephys_data.collection[recording].event_dict = timestamp_dicts[recording_key_without_suffix]
        
        # Extract the subject from the recording key
        start = recording.find('subj_') + 5  # Start index after 'subj_'
        subject = recording[start:start+3]
        
        # Assign the extracted subject
        ephys_data.collection[recording].subject = subject

In [10]:
spike_analysis = spike.SpikeAnalysis_MultiRecording(ephys_data, timebin = 1, ignore_freq = 0.5)

These recordings are missing event dictionaries:
['20230628_111202_standard_comp_to_novel_agent_D1_subj_1-1vs1-2and2-2_merged.rec', '20230628_111202_standard_comp_to_novel_agent_D1_subj_1-2vs1-1and2-1_merged.rec', '20230629_111937_standard_comp_to_novel_agent_D2_subj_1-1v1-4and2-1_merged.rec', '20230629_111937_standard_comp_to_novel_agent_D2_subj_1-4vs1-1and2-2_merged.rec', '20230630_115506_standard_comp_to_novel_agent_D3_subj_1-4vs1-2and2-1_merged_merged.rec']
These recordings are missing subjects: ['20230628_111202_standard_comp_to_novel_agent_D1_subj_1-1vs1-2and2-2_merged.rec', '20230628_111202_standard_comp_to_novel_agent_D1_subj_1-2vs1-1and2-1_merged.rec', '20230629_111937_standard_comp_to_novel_agent_D2_subj_1-1v1-4and2-1_merged.rec', '20230629_111937_standard_comp_to_novel_agent_D2_subj_1-4vs1-1and2-2_merged.rec', '20230630_115506_standard_comp_to_novel_agent_D3_subj_1-4vs1-2and2-1_merged_merged.rec']


In [11]:
recordings = spike_analysis.ephyscollection.collection
recording_name = '20230622_110832_standard_comp_to_both_rewarded_D1_subj_1-1_t1b3L_box1_merged.rec'
recording1 = recordings.get(recording_name)
recording1.event_dict

AttributeError: 'NoneType' object has no attribute 'event_dict'